In [2]:
import numpy as np
import random
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Flatten
from keras import layers
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import keras
import tensorflow as tf

2022-11-29 05:45:22.921687: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


# Check GPU & Set Seed 

In [18]:
tf.config.list_physical_devices('GPU')

seed = 2345
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)


# Loading Data

In [91]:
import pickle

with open("Data/utfac_normal.np", "rb") as f:
    train_normal = pickle.load(f)

with open("Data/utfac_osa.np", "rb") as f:
    train_osa = pickle.load(f)

with open("Data/xyz_1_71", "rb") as f:
    xyz_normal = pickle.load(f)
    
with open("Data/xyz_osa", "rb") as f:
    xyz_osa = pickle.load(f)
# read ecg
# with open("", "rb") as f:
#     train_ecg = pickle.load(f)

# Preprocess ECG

In [87]:
with open("Data/best-of-best-ecg-normal", "rb") as f:
    normal_ecg = pickle.load(f)
with open("Data/best-of-best-ecg-osa", "rb") as f:
    osa_ecg = pickle.load(f)

In [88]:
normal_target = normal_ecg
osa_target = osa_ecg

In [89]:
i = 1
cnt_all = 0
store_normal_ecg = []

for person in normal_target:
    i += 1
    length = len(person)
    divide_len = (int)(length/10)
    cnt = 0
    for k in range(0, length - divide_len -1, divide_len):
        std = np.std(person[k:k+divide_len])
        mean = np.mean(person[k:k+divide_len])
        for j in range(k, k + divide_len):
            if abs(person[j] - person[j+1]) > 2.4 * std:
                cnt += 1
    if (cnt / length > 0.1):
        cnt_all+=1
    store_normal_ecg.append(cnt / length)
print(cnt_all)

13


In [90]:
i = 1
cnt_all = 0
store_osa_ecg = []

for person in osa_target:
    i += 1
    length = len(person)
    divide_len = (int)(length/10)
    cnt = 0
    for k in range(0, length - divide_len - 1, divide_len):
        std = np.std(person[k:k+divide_len])
        mean = np.mean(person[k:k+divide_len])
        for j in range(k, k + divide_len):
            if abs(person[j] - person[j+1]) > 2.4 * std:
                cnt += 1
    if (cnt / length > 0.1):
        cnt_all+=1
    store_osa_ecg.append(cnt / length)
print(cnt_all)

53


# Data preprocessing

In [92]:
import random
osa_mean = np.mean(train_osa)

# if ecg is big and motion is small, make sound bigger
a = 0
for idx in range(0, 71):
    if (store_normal_ecg[idx] > 0.1) & (xyz_normal[idx] < 0.012):
        for n in range(0, len(train_normal[idx])):
            train_normal[idx][n] = osa_mean + random.randrange(12, 20)
print(a)
a = 0
for idx in range(0, 229):
    if (store_osa_ecg[idx] > 0.1) & (xyz_osa[idx] < 0.012):
        a +=1
        for n in range(0, len(train_osa[idx])):
            train_osa[idx][n] = osa_mean + random.randrange(12, 20)
print(a)        


0
32


# Data Dividing

In [49]:
np_normal = np.array(train_normal)
np_osa = np.array(train_osa)

num_n = 71
num_a = 229

num_train_n = int(num_n/5) * 4
num_train_a = int(num_a/5) * 4

x_train = np.concatenate([np_normal[:num_train_n], np_normal[:num_train_n] , np_normal[:num_train_n], np_osa[:num_train_a]])
x_test = np.concatenate([np_normal[num_train_n:], np_normal[num_train_n:], np_normal[num_train_n:], np_osa[num_train_a:]])

y_train = np.concatenate([np.zeros(num_train_n * 3), np.ones(num_train_a)])
y_test = np.concatenate([np.zeros((num_n - num_train_n) * 3), np.ones(num_a - num_train_a)])

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# Data Reshaping

In [94]:
x_train = x_train.reshape(x_train.shape[0], 20, 1)
x_test = x_test.reshape(x_test.shape[0], 20, 1)

# Model Training

In [120]:
with tf.device('device:GPU:0'):
    model = Sequential()
    model.add(layers.Conv1D(filters = 64, kernel_size = 5, padding='same', activation='relu', input_shape = (20, 1)))
    model.add(layers.AveragePooling1D())
    model.add(layers.Dropout(0.6))
    model.add(layers.Conv1D(64, 5, padding='same', activation='relu'))
    model.add(layers.AveragePooling1D())
    model.add(layers.Dropout(0.6))
    model.add(Flatten())
    model.add(layers.Dense(2, activation='sigmoid'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_18 (Conv1D)           (None, 20, 64)            384       
_________________________________________________________________
average_pooling1d_18 (Averag (None, 10, 64)            0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 10, 64)            20544     
_________________________________________________________________
average_pooling1d_19 (Averag (None, 5, 64)             0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 5, 64)             0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 320)              

In [121]:
import keras

model.compile(optimizer=Adam(lr=0.001),
              loss='binary_crossentropy',
              metrics=['acc'])

callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                                           patience=300),
             keras.callbacks.ModelCheckpoint(filepath='b_model3.h5',
                                             monitor='val_loss',
                                             save_best_only=True)]

history = model.fit(x_train, y_train,
                    epochs=400,
                    batch_size=8,
                    validation_data=(x_test, y_test),
                    callbacks = callbacks)

Epoch 1/400
30/30 [==============================] - 1s 11ms/step - loss: 0.9654 - acc: 0.7070 - val_loss: 0.4920 - val_acc: 0.7656
Epoch 2/400
30/30 [==============================] - 0s 6ms/step - loss: 0.6292 - acc: 0.7692 - val_loss: 0.4881 - val_acc: 0.7656
Epoch 3/400
30/30 [==============================] - 0s 4ms/step - loss: 0.6040 - acc: 0.7343 - val_loss: 0.4821 - val_acc: 0.7656
Epoch 4/400
30/30 [==============================] - 0s 6ms/step - loss: 0.5072 - acc: 0.7530 - val_loss: 0.4751 - val_acc: 0.7656
Epoch 5/400
30/30 [==============================] - 0s 5ms/step - loss: 0.6128 - acc: 0.6989 - val_loss: 0.4860 - val_acc: 0.7656
Epoch 6/400
30/30 [==============================] - 0s 5ms/step - loss: 0.5112 - acc: 0.7523 - val_loss: 0.4881 - val_acc: 0.7656
Epoch 7/400
30/30 [==============================] - 0s 3ms/step - loss: 0.4672 - acc: 0.7704 - val_loss: 0.5229 - val_acc: 0.7656
Epoch 8/400
30/30 [==============================] - 0s 3ms/step - loss: 0.4598 - 

Epoch 125/400
30/30 [==============================] - 0s 5ms/step - loss: 0.3524 - acc: 0.8392 - val_loss: 0.4366 - val_acc: 0.7969
Epoch 126/400
30/30 [==============================] - 0s 4ms/step - loss: 0.3584 - acc: 0.8532 - val_loss: 0.4305 - val_acc: 0.8125
Epoch 127/400
30/30 [==============================] - 0s 4ms/step - loss: 0.2970 - acc: 0.8494 - val_loss: 0.4243 - val_acc: 0.8125
Epoch 128/400
30/30 [==============================] - 0s 5ms/step - loss: 0.3309 - acc: 0.8308 - val_loss: 0.4295 - val_acc: 0.7969
Epoch 129/400
30/30 [==============================] - 0s 5ms/step - loss: 0.2935 - acc: 0.8667 - val_loss: 0.4568 - val_acc: 0.7969
Epoch 130/400
30/30 [==============================] - 0s 5ms/step - loss: 0.3516 - acc: 0.8165 - val_loss: 0.4484 - val_acc: 0.7969
Epoch 131/400
30/30 [==============================] - 0s 6ms/step - loss: 0.3337 - acc: 0.8610 - val_loss: 0.4689 - val_acc: 0.7656
Epoch 132/400
30/30 [==============================] - 0s 4ms/step - 

Epoch 248/400
30/30 [==============================] - 0s 5ms/step - loss: 0.2721 - acc: 0.8970 - val_loss: 0.3917 - val_acc: 0.8594
Epoch 249/400
30/30 [==============================] - 0s 5ms/step - loss: 0.2733 - acc: 0.8845 - val_loss: 0.4360 - val_acc: 0.7969
Epoch 250/400
30/30 [==============================] - 0s 3ms/step - loss: 0.2931 - acc: 0.8578 - val_loss: 0.4180 - val_acc: 0.8281
Epoch 251/400
30/30 [==============================] - 0s 4ms/step - loss: 0.2490 - acc: 0.9111 - val_loss: 0.4035 - val_acc: 0.8594
Epoch 252/400
30/30 [==============================] - 0s 4ms/step - loss: 0.2735 - acc: 0.8660 - val_loss: 0.4436 - val_acc: 0.8125
Epoch 253/400
30/30 [==============================] - 0s 8ms/step - loss: 0.2816 - acc: 0.8958 - val_loss: 0.4263 - val_acc: 0.7969
Epoch 254/400
30/30 [==============================] - 0s 4ms/step - loss: 0.2699 - acc: 0.8483 - val_loss: 0.4090 - val_acc: 0.8438
Epoch 255/400
30/30 [==============================] - 0s 3ms/step - 

Epoch 371/400
30/30 [==============================] - 0s 3ms/step - loss: 0.2305 - acc: 0.9018 - val_loss: 0.4002 - val_acc: 0.8594
Epoch 372/400
30/30 [==============================] - 0s 4ms/step - loss: 0.2997 - acc: 0.8726 - val_loss: 0.3970 - val_acc: 0.8750
Epoch 373/400
30/30 [==============================] - 0s 6ms/step - loss: 0.2069 - acc: 0.9250 - val_loss: 0.4501 - val_acc: 0.7969
Epoch 374/400
30/30 [==============================] - 0s 4ms/step - loss: 0.2318 - acc: 0.8909 - val_loss: 0.4103 - val_acc: 0.7969
Epoch 375/400
30/30 [==============================] - 0s 4ms/step - loss: 0.2616 - acc: 0.9264 - val_loss: 0.3968 - val_acc: 0.8125
Epoch 376/400
30/30 [==============================] - 0s 4ms/step - loss: 0.2512 - acc: 0.8654 - val_loss: 0.3860 - val_acc: 0.8594
Epoch 377/400
30/30 [==============================] - 0s 5ms/step - loss: 0.2286 - acc: 0.9065 - val_loss: 0.4257 - val_acc: 0.7969
Epoch 378/400
30/30 [==============================] - 0s 4ms/step - 

# Test

## Test Data Load

In [1]:
import pickle

with open("Data/Private_Test_sound1", "rb") as f:
    test_np = pickle.load(f)

with open("Data/Private_Test_sound2", "rb") as f:
    tmp_np = pickle.load(f)
    test_np.extend(tmp_np)

with open("Data/Private_Test_sound3", "rb") as f:
    tmp_np = pickle.load(f)
    test_np.extend(tmp_np)
    
with open("Data/Private_Test_sound4", "rb") as f:
    tmp_np = pickle.load(f)
    test_np.extend(tmp_np)
    
with open("Data/best-of-best-ecg-private-test", "rb") as f:
    test_ecg = pickle.load(f)

with open("Data/Private_Test_xyz", "rb") as f:
    test_xyz = pickle.load(f)

## Test Data preprocessing

In [3]:
i = 1
cnt_all = 0
store_test_ecg = []

for person in test_ecg:
    i += 1
    length = len(person)
    divide_len = (int)(length/10)
    cnt = 0
    for k in range(0, length - divide_len -1, divide_len):
        std = np.std(person[k:k+divide_len])
        mean = np.mean(person[k:k+divide_len])
        for j in range(k, k + divide_len):
            if abs(person[j] - person[j+1]) > 2.4 * std:
                cnt += 1
    if (cnt / length > 0.1):
        cnt_all+=1
    store_test_ecg.append(cnt / length)
print(cnt_all)

0


In [6]:
import random

a = 0
for idx in range(0, 100):
    if (store_test_ecg[idx] > 0.1) & (test_xyz[idx] < 0.012):
        a +=1
        for n in range(0, len(test_np[idx])):
            test_np[idx][n] = osa_mean + random.randrange(12, 20)
print(a)

0


In [7]:
test_np = np.array(test_np)
test_np = test_np.reshape(test_np.shape[0], 20, 1)
np.shape(test_np)

(100, 20, 1)

## Trained Model Load

In [8]:
saved_model = tf.keras.models.load_model('Saved_Model.h5')
output = saved_model.predict(test_np)

2022-11-29 05:46:06.911795: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-29 05:46:06.913614: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-29 05:46:06.943246: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:45:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.54GiB deviceMemoryBandwidth: 715.34GiB/s
2022-11-29 05:46:06.943284: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-11-29 05:46:06.951349: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-11-29 05:46:06.951420: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-1

# Make Answer Sheet

In [9]:
a = []
for i in output:
    a.append("Normal" if i[0] > i[1] else "OSA")
a.count("OSA")

48

In [12]:
answer = []
for i in range(1, 101):
    answer.append(["Private_Test_" + str(i).zfill(3), a[i-1]])

In [13]:
import csv

with open("Private_Infer_Result_2345.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(answer)
    f.close()

In [129]:
# !curl --user "inticoy0406@gmail.com:!2345Maic"-X POST -F "files=@submission_jukim_and_gyoon_and_choi22.csv" https://maic.or.kr/api/direct-report/596?password=aPfnF7TsAuMYdwKd

curl: (6) Could not resolve host: POST
{"message":"2345팀 제출이 완료되었습니다. 남은 일일 제출 횟수는 3회/10회 입니다."}